## Ejercicio 22
Top 5 de lenguajes usados por usuarios bilingües. 

## Preparando sesión de Spark

### Spark session y Spark context

In [1]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

## Lectura de archivo

In [2]:
#import pandas as pd
import numpy as np 
#texts = pd.read_csv('./WikipediaDataset/languages.csv', usecols = ["babel_user", "babel_lang"])
#texts.to_parquet('./WikipediaDataset/languagesSinNivel.parquet')
#del texts
#Dataset de languagesSinNivel.parquet fue creado con las líneas anteriores, y ahora puede ser 
#leído de la siguiente forma:
dataframeLenguajes = sqlContext.read.parquet('./WikipediaDataset/languagesSinNivel.parquet')
rddLenguajes = dataframeLenguajes.rdd

## Preparando y limpiando el dataset

In [3]:
rddLenguajes

MapPartitionsRDD[7] at javaToPython at NativeMethodAccessorImpl.java:0

In [4]:
rddLenguajes1 = rddLenguajes

In [5]:
rddLenguajes1.take(5)

[Row(babel_user=4502458, babel_lang='FR'),
 Row(babel_user=5928200, babel_lang='aa'),
 Row(babel_user=46918, babel_lang='ab'),
 Row(babel_user=2050449, babel_lang='ab'),
 Row(babel_user=4715583, babel_lang='ace')]

In [6]:
rddLenguajes1.count()

35666

In [7]:
rddLenguajes1 = rddLenguajes1.filter(lambda x: ((x[0] != None) and (x[1] != None)))
rddLenguajes1.count()

35662

## Desarrollo

In [8]:
rddLenguajesPorUsuario = rddLenguajes1.map(lambda x: (x[0], 1)).reduceByKey(lambda x, y: x + y)
rddLenguajesPorUsuario.take(5)

[(4502458, 3), (5928200, 190), (46918, 3), (2050449, 7), (4715583, 30)]

In [9]:
rddUsuariosBilingues = rddLenguajesPorUsuario.filter(lambda x: (x[1] == 2))
rddUsuariosBilingues.take(5)

[(1801431, 2), (1724336, 2), (1725400, 2), (674716, 2), (4334792, 2)]

In [10]:
rddUsuariosBilingues.count()

3136

In [11]:
rddUsuariosBilinguesConLenguajes = rddUsuariosBilingues.join(rddLenguajes1)
rddUsuariosBilinguesConLenguajes.take(5)

[(1724336, (2, 'ang')),
 (1724336, (2, 'es')),
 (1725400, (2, 'ang')),
 (1725400, (2, 'es')),
 (674716, (2, 'ar'))]

In [12]:
rddLenguajesUsadosPorBilingues = rddUsuariosBilinguesConLenguajes.map(lambda x: (x[1][1], 1))
rddLenguajesUsadosPorBilingues.take(5)

[('ang', 1), ('es', 1), ('ang', 1), ('es', 1), ('ar', 1)]

In [13]:
rddLenguajesUsadosPorBilingues = rddLenguajesUsadosPorBilingues.reduceByKey(lambda x, y: x + y)
rddLenguajesUsadosPorBilingues.take(5)

[('ar', 2), ('ca', 54), ('en', 2944), ('da', 2), ('hu', 1)]

In [14]:
print(rddLenguajesUsadosPorBilingues.takeOrdered(5, lambda x: -x[1]))

[('es', 3054), ('en', 2944), ('fr', 56), ('ca', 54), ('pt', 22)]


## Conclusión
De las 3136 personas que afirman ser bilingues (con cualquier nivel en babel), 3054 tienen cierto nivel de español mientras que 2944 tienen cierto nivel de inglés. Mucho más atrás, el tercer idioma más hablado por bilingües es el frances, con 56 personas, seguido por 54 personas que hablan catalán, y por último el portugues, con 22 hablantes.

## En limpio
A lo largo de la consigna se fue descubriendo los rdds para ir verificando cuáles operaciones hacer y que todo funcione correctamente. Con esto en mente está claro que la consigna puede ser resuelta con menos líneas de código, por lo que se procede a mostrar una solución 'final', con los rdds recién leídos y cargados, saltándose los pasos intermedios de investigación de los rdds.

Las siguientes transformaciones y acciones están hechas sobre el rdds original obtenido al hacer la lectura del archivo correspondiente.

In [15]:
rddUsuariosBilingues = rddLenguajes.filter(lambda x: ((x[0] != None) and (x[1] != None)))\
.map(lambda x: (x[0], 1)).reduceByKey(lambda x, y: x + y).filter(lambda x: (x[1] == 2))

In [16]:
print(rddUsuariosBilingues.join(rddLenguajes).map(lambda x: (x[1][1], 1)).reduceByKey(lambda x, y: x + y)\
      .takeOrdered(5, lambda x: -x[1]))

[('es', 3054), ('en', 2944), ('fr', 56), ('ca', 54), ('pt', 22)]
